## Updating and Deleting Rows with Core

So far we've covered `Insert`, so that we can get some data into our database, and then spent a lot of time on `Select` which _handles the broad range of usage patterns_ used for __retrieving data from the database__. In this section we will cover the `Update` and `Delete` constructs, which are used to __modify existing rows__ as well as __delete existing rows__. This section will cover these constructs from a _Core-centric perspective_.

> `ORM Readers` - As was the case mentioned at `Inserting Rows with Core`, the `Update` and `Delete` operations when used _with the ORM_ are usually __invoked internally from the Session object__ as part of the unit of work process.
>
> However, __unlike Insert__, the `Update` and `Delete` constructs __can also be used directly with the ORM__, using a pattern known as __`"ORM-enabled update and delete"`__; for this reason, familiarity with these constructs is useful for ORM use. Both styles of use are discussed in the sections `Updating ORM Objects` and `Deleting ORM Objects`.

In [15]:
from sqlalchemy import (
    MetaData, Table, Column, Integer, String, ForeignKey,
    create_engine, select, insert, update, bindparam,
)
from sqlalchemy.orm import registry, relationship
from sqlalchemy.dialects import mysql

In [2]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
metadata_obj = MetaData()

In [3]:
user_table = Table(
    "user_account", metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

address_table = Table(
    "address", metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False)
)

metadata_obj.create_all(engine)

2022-10-05 13:18:35,865 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-05 13:18:35,866 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-10-05 13:18:35,867 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-05 13:18:35,869 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-10-05 13:18:35,870 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-05 13:18:35,872 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-10-05 13:18:35,874 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-05 13:18:35,875 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-10-05 13:18:35,876 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-05 13:18:35,878 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-10-05 13:18:35,878 INFO sqlalchemy.engine.Engine [no key 0.00084s] ()
2022-10-05 13:18:35,882 INFO sqlalchemy.engine.Engine 
C

In [4]:
stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"}
        ]
    )
    conn.commit()

2022-10-05 13:18:35,958 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-05 13:18:35,960 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-10-05 13:18:35,961 INFO sqlalchemy.engine.Engine [generated in 0.00318s] ('spongebob', 'Spongebob Squarepants')
2022-10-05 13:18:35,964 INFO sqlalchemy.engine.Engine COMMIT
2022-10-05 13:18:35,968 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-05 13:18:35,968 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-10-05 13:18:35,969 INFO sqlalchemy.engine.Engine [generated in 0.00187s] (('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star'))
2022-10-05 13:18:35,971 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
scalar_subq = (
    select(user_table.c.id).
    where(user_table.c.name == bindparam("username")).
    scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {"username": 'spongebob', "email_address": "spongebob@sqlalchemy.org"},
            {"username": 'sandy', "email_address": "sandy@sqlalchemy.org"},
            {"username": 'sandy', "email_address": "sandy@squirrelpower.org"},
        ]
    )
    conn.commit()

2022-10-05 13:18:36,050 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-05 13:18:36,052 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)
2022-10-05 13:18:36,053 INFO sqlalchemy.engine.Engine [generated in 0.00307s] (('spongebob', 'spongebob@sqlalchemy.org'), ('sandy', 'sandy@sqlalchemy.org'), ('sandy', 'sandy@squirrelpower.org'))
2022-10-05 13:18:36,055 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
mapped_registry = registry()
Base = mapped_registry.generate_base()

In [7]:
class User(Base):
    __tablename__ = "user_account"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    
    addresses = relationship("Address", back_populates="user")
    
    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

In [8]:
class Address(Base):
    __tablename__ = "address"
    
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"))
    
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [9]:
mapped_registry.metadata.create_all(engine)

2022-10-05 13:18:36,346 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-05 13:18:36,348 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-10-05 13:18:36,349 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-05 13:18:36,350 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-10-05 13:18:36,352 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-05 13:18:36,354 INFO sqlalchemy.engine.Engine COMMIT


#### The `update()` SQL Expression Construct

The `update()` function generates a _new instance of Update_ which represents an `UPDATE` statement in SQL, that will __update existing data in a table__. Like the `insert()` construct, there is a `"traditional"` form of `update()`, which __emits `UPDATE` against a single table__ at a time and __`does not` return any rows__. However _some backends_ support an `UPDATE` statement that __may modify multiple tables at once__, and the `UPDATE` statement also supports `RETURNING` such that _columns contained in matched rows may be returned in the result set_.

In [10]:
stmt = (
    update(user_table).where(user_table.c.name == "patrick").
    values(fullname="Patrick the Star")
)
print(stmt)

UPDATE user_account SET fullname=:fullname WHERE user_account.name = :name_1


The `Update.values()` method __controls the contents of the SET elements__ of the `UPDATE` statement. This is the __same method shared by the `Insert` construct__. Parameters can normally be _passed using the column names as keyword arguments_. `UPDATE` _supports all the major SQL forms_ of `UPDATE`, __including updates against expressions__, where we can make use of `Column` expressions.

In [11]:
stmt = update(user_table).values(fullname="Username: " + user_table.c.name)
print(stmt)

UPDATE user_account SET fullname=(:name_1 || user_account.name)


To support `UPDATE` in an __`"executemany"`__ context, where many parameter sets will be invoked against the same statement, the `bindparam()` construct may be used to __set up bound parameters__; these __replace the places__ that _literal values would normally go_.

In [12]:
stmt = (
    update(user_table).
    where(user_table.c.name == bindparam("oldname")).
    values(name=bindparam("newname"))
)

with engine.connect() as conn:
    conn.execute(
        stmt,
        [
            {'oldname':'jack', 'newname':'ed'},
            {'oldname':'wendy', 'newname':'mary'},
            {'oldname':'jim', 'newname':'jake'},
        ]
    )

2022-10-05 13:18:36,606 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-05 13:18:36,609 INFO sqlalchemy.engine.Engine UPDATE user_account SET name=? WHERE user_account.name = ?
2022-10-05 13:18:36,610 INFO sqlalchemy.engine.Engine [generated in 0.00336s] (('ed', 'jack'), ('mary', 'wendy'), ('jake', 'jim'))
2022-10-05 13:18:36,611 INFO sqlalchemy.engine.Engine ROLLBACK


##### Correlated Updates

An `UPDATE` statement can _make use of rows in other tables_ by using a __correlated subquery__. A `subquery` may be used _anywhere a column expression might be placed_.

In [13]:
scalar_subq = (
    select(address_table.c.email_address).
    where(address_table.c.user_id == user_table.c.id).
    order_by(address_table.c.id).
    limit(1).scalar_subquery()
)

update_stmt = update(user_table).values(fullname=scalar_subq)
print(update_stmt)

UPDATE user_account SET fullname=(SELECT address.email_address 
FROM address 
WHERE address.user_id = user_account.id ORDER BY address.id
 LIMIT :param_1)


##### UPDATE ... FROM

_Some databases_ such as `PostgreSQL` and `MySQL` support a syntax `"UPDATE FROM"` where _additional tables may be stated directly_ in a __special FROM clause__. This syntax will be __generated implicitly__ when additional tables are located in the `WHERE` clause of the statement.

In [14]:
update_stmt = (
    update(user_table).
    where(user_table.c.id == address_table.c.user_id).
    where(address_table.c.email_address == "patrick@aol.com").
    values(fullname="Pat")
)
print(update_stmt)

UPDATE user_account SET fullname=:fullname FROM address WHERE user_account.id = address.user_id AND address.email_address = :email_address_1


There is also a `MySQL` specific syntax that __can `UPDATE` multiple tables__. This __requires__ we refer to `Table` objects in the `VALUES` clause in order to _refer to additional tables_.

In [16]:
update_stmt = (
    update(user_table).
    where(user_table.c.id == address_table.c.user_id).
    where(address_table.c.email_address == "patrick@aol.com").
    values(
        {
            user_table.c.fullname: "Pat",
            address_table.c.email_address: "pat@aol.com",
        }
    )
)
print(update_stmt.compile(dialect=mysql.dialect()))

UPDATE user_account, address SET address.email_address=%s, user_account.fullname=%s WHERE user_account.id = address.user_id AND address.email_address = %s
